In [2]:
import torch
from pointnet.model import PointNet

model = PointNet(l=10,batch_norm=True)
model.load_state_dict(torch.load("logs/100k top_variance, with_alpha, camera_loss only/model.pt"))
model.eval()

/home/niedermayr/anaconda3/envs/punctum12/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PointNet(
  (feat): FeatureNet(
    (conv1): Conv1d(7, 32, kernel_size=(1,), stride=(1,), bias=False)
    (conv2): Conv1d(32, 128, kernel_size=(1,), stride=(1,), bias=False)
    (conv3): Conv1d(128, 256, kernel_size=(1,), stride=(1,), bias=False)
    (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (fc1): Linear(in_features=256, out_features=128, bias=False)
  (fc2): Linear(in_features=128, out_features=64, bias=False)
  (fc3): Linear(in_features=64, out_features=484, bias=True)
  (dropout): Dropout(p=False, inplace=False)
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [1]:
from pointnet.dataset import OctantDataset

ds = OctantDataset("datasets/neuschwanstein/octants_1024max_sh",preload=True)

ds

/home/niedermayr/anaconda3/envs/punctum12/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
loading dataset: 100%|██████████| 1612599/1612599 [2:31:21<00:00, 177.58it/s]      


In [2]:
from pointnet.model import PointNet
import torch

model = PointNet(10,4,batch_norm=True,use_dropout=True)
model.eval()
model.load_state_dict(torch.load("logs/100k top_variance, with_alpha, camera_loss only, small dropout/model.pt"))

traced_model = torch.jit.trace_module(model,{
    "forward":[torch.rand((1,3)),torch.rand((1,4)),torch.zeros(1,dtype=torch.long)]
})

torch.jit.save(traced_model,"traced_model.pt")

/home/niedermayr/punctum/pointnet/model.py:16: UserWarning: scatter_reduce() is in beta and the API may change at any time. (Triggered internally at  /opt/conda/conda-bld/pytorch_1656352645774/work/aten/src/ATen/native/TensorAdvancedIndexing.cpp:1550.)
  return target.scatter_reduce(0, batch_idx, x, reduce="amax")


In [3]:
model =torch.jit.load("logs/100k top_variance l=5/traced_model.pt").cuda()
model.eval()

RecursiveScriptModule(
  original_name=PointNet
  (feat): RecursiveScriptModule(
    original_name=FeatureNet
    (conv1): RecursiveScriptModule(original_name=Conv1d)
    (conv2): RecursiveScriptModule(original_name=Conv1d)
    (conv3): RecursiveScriptModule(original_name=Conv1d)
    (bn1): RecursiveScriptModule(original_name=BatchNorm1d)
    (bn2): RecursiveScriptModule(original_name=BatchNorm1d)
    (bn3): RecursiveScriptModule(original_name=BatchNorm1d)
  )
  (fc1): RecursiveScriptModule(original_name=Linear)
  (fc2): RecursiveScriptModule(original_name=Linear)
  (fc3): RecursiveScriptModule(original_name=Linear)
  (dropout): RecursiveScriptModule(original_name=Dropout)
  (bn1): RecursiveScriptModule(original_name=BatchNorm1d)
  (bn2): RecursiveScriptModule(original_name=BatchNorm1d)
)

In [8]:
model(torch.rand((1,3)).cuda(),torch.rand((1,4)).cuda(),torch.zeros(1,dtype=torch.long).cuda())

tensor([[[ 2.2746e+02,  1.5056e+02,  1.3226e+02,  6.9351e+01],
         [ 9.5928e+01,  8.8738e+01,  6.5791e+01, -1.3887e+00],
         [-9.0768e+01, -9.3234e+01, -8.0737e+01, -1.5213e+00],
         [ 6.4212e+01,  6.3021e+01,  4.8788e+01,  3.3735e+00],
         [-6.6433e+01, -6.3780e+01, -5.2144e+01, -1.4481e+01],
         [ 2.8225e+01,  3.3153e+01,  3.8776e+01, -1.7906e+01],
         [-4.5822e+01, -4.2476e+01, -3.0700e+01,  4.3266e+01],
         [-7.1480e+01, -7.2470e+01, -7.8285e+01, -1.5665e+01],
         [-7.7783e+01, -7.5159e+01, -6.6739e+01, -6.1593e+01],
         [-6.4490e+01, -6.9650e+01, -7.1858e+01, -9.1307e-01],
         [ 5.3329e+01,  5.7251e+01,  6.2739e+01,  1.5869e-01],
         [ 1.8775e+02,  1.8125e+02,  1.7560e+02,  6.1169e-01],
         [-1.2690e+02, -1.2612e+02, -1.2807e+02, -5.6530e-01],
         [-9.5310e+01, -9.3936e+01, -9.1858e+01,  1.4232e+00],
         [ 6.5026e+01,  6.9402e+01,  7.7273e+01,  8.6219e-01],
         [-3.9224e+01, -3.7884e+01, -3.8269e+01,  1.429

In [4]:
from tqdm import tqdm
from pointnet.pointclouds import collate_batched

with open("datasets/neuschwanstein/octants_1024max_sh/coefs.raw","wb") as f:
    batches = torch.arange(len(ds)).split(256)
    for batch in tqdm(batches):
        pc = collate_batched([ds[i][0] for i in batch])
        coefs = model(pc.points_packed().cuda(),pc.features_packed().cuda(),pc.packed_to_cloud_idx().cuda())
        octant_id = int(pc.file_names[0].split("/")[-1].split(".")[0].split("_")[1])
        f.write(octant_id.to_bytes(8,"little"))
        f.write(coefs.detach().flatten().cpu().numpy().tobytes())

100%|██████████| 6300/6300 [09:20<00:00, 11.24it/s]
